# Offshore wind farms

This notebook pulls wind farm data from OpenStreetMap (OSM) and plots them in Bokeh plots. The idea is to produce something interactive and fun to play with. 

## Setup

Import libraries and standard tools used

In [148]:
import os
import requests
import json

import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource, OpenURL, TapTool
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.palettes import viridis, Category20

from pyproj import Transformer

output_notebook()

Loading BokehJS ...

In [149]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
area_UK = "(49.325, -11.25, 59.356, 1.95)"
area = "(51.4,2.6,51.9,3.2)" # Borselle etc.
area = "(51.55,1.4,51.75,1.6)" # London Array
area = "(40,-20,90,20)" # Europe
area = "(-90,-180,90,180)" # Europe


In [151]:
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

In [152]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")
 

In [153]:
# Place to save data so that the number of OSM requests are reduced when re-loading the same data
path_save = r"data/"

if not os.path.exists(path_save):
    os.makedirs(path_save)
    

In [154]:
def get_wind_farms_rel(area="(55.503, 37.0789,56.003, 38.1871)"):

    key = "seamark:production_area:category"
    tag = "wind_farm"
    output = "geom"
    recursion = ""
    element = "rel"

    data = osm.toolkit.get_osm_data(key=key,
                                    tag=tag,
                                    area=area,
                                    output=output,
                                    recursion=recursion,
                                    element=element)
    
    
    return data

In [155]:
def get_wind_farms_way(area="(55.503, 37.0789,56.003, 38.1871)"):

    key = "seamark:production_area:category"
    tag = "wind_farm"
    output = "geom"
    recursion = ""
    element = "way"

    data = osm.toolkit.get_osm_data(key=key,
                                    tag=tag,
                                    area=area,
                                    output=output,
                                    recursion=recursion,
                                    element=element)
    
    
    return data

In [156]:
def get_wind_farm_data(area,name):
    file_name = path_save+name+"_data.pkl"

    if os.path.isfile(file_name):
        df = pd.read_pickle(file_name)
    else:
        df_rel = get_wind_farms_rel(area)
        df_way = get_wind_farms_way(area)

        df = pd.concat([df_rel,df_way])
        
        df.to_pickle(file_name)
        
    return df

In [213]:
def plot_wind_farms(df_wind_farms,kwargs_for_figure={}):

    # Define default and then update figure and marker options based on kwargs
    figure_options = {
        "tools": "save,pan,wheel_zoom,reset,help",
        "x_axis_label": "Longitude",
        "y_axis_label": "Latitude",
        "match_aspect": True,
    }
    figure_options.update(kwargs_for_figure)

    # Create a bokeh figure with tiles
    p = figure(
        width=800,
        height=800,
        x_axis_type="mercator",
        y_axis_type="mercator",
        **figure_options,
    )

    p.add_tile(MAP_TILES['OpenMap'])

    xx = list()
    yy = list()
    names = list()
    names_en = list()
    capacity = list()
    nodes = list()
    colors = list()
    for cnt,row in df_wind_farms.iterrows():

        try:
            for way in row["members"]:
                xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(way['geometry'])['lat']),
                                                    list(pd.DataFrame.from_records(way['geometry'])['lon']))
            
                xx.append(xxx)
                yy.append(yyy)
                nodes.append(row['id'])
                names.append(row['name'])
                names_en.append(row['name_en'])
                capacity.append(row['plant_output_electricity'])
                colors.append(row['color'])
            
        except:
            xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(row['geometry'])['lat']),
                                                    list(pd.DataFrame.from_records(row['geometry'])['lon']))

            xx.append(xxx)
            yy.append(yyy)
            nodes.append(row['id'])
            names.append(row['name'])
            names_en.append(row['name_en'])
            capacity.append(row['plant_output_electricity'])
            colors.append(row['color'])

    source = ColumnDataSource({'xs':xx,'ys':yy,'color':colors,'name':names,'capacity':capacity,'id':nodes})

    render_lines = p.multi_line('xs',
                 'ys',
                 source=source,
                 line_width=2,
                 line_color='color',
          )

    # HoverTool for power lines
    hover_lines = HoverTool(renderers=[render_lines])
    hover_lines.tooltips = [("Name", "@name"),("Capacity", "@capacity")]
    p.add_tools(hover_lines)

    url = "https://www.openstreetmap.org/way/@id/"
    
    render_lines.selection_glyph = None
    render_lines.nonselection_glyph = None
    taptool_wind_turbines = TapTool(renderers=[render_lines])
    taptool_wind_turbines.callback = OpenURL(url=url)
    p.add_tools(taptool_wind_turbines)

    
    return p

In [214]:
import re

df_wind_farms = get_wind_farm_data(area,"wind_farms")

df_wind_farms = df_wind_farms.rename(columns=lambda x: re.sub(':','_',x))

df_wind_farms["color"] = "blue"


In [215]:
output_file("wind_farms.html")

In [216]:
wind_farms = plot_wind_farms(df_wind_farms)



In [231]:
from bokeh.models import Range1d

x,y = TRANSFORM_4326_TO_3857.transform([53.5,56],[5.5,9])

# set a range using a Range1d
wind_farms.x_range = Range1d(x[0], x[1])
wind_farms.y_range = Range1d(y[0], y[1])

wind_farms.title.text = "Offshore wind farms in OpenStreetMap"

In [232]:
show(wind_farms)

## UK rivers coloured by name

In [233]:
# uk_colors.add_tools(HoverTool(tooltips=[("River","@name")]))